# Imports + Config

In [1]:
import json
import requests
import pandas as pd
!pip install --upgrade typing_extensions
!pip install pydantic
from pydantic import BaseModel

# Ollama Config
OLLAMA_NODE = "arctrdgn001"
BASE_URL = f"http://{OLLAMA_NODE}:11434/api/chat"

model = "gemma3-optimized:27b"

#  Grabbing DAIC-WOZ data (Data Loader)

In [2]:
test_transcript = pd.read_csv(r"/data/users4/xli/ai-psychiatrist/datasets/daic_woz_dataset/305_P/305_TRANSCRIPT.csv", sep="\t")
train_phq8_dataset = pd.read_csv(r"/data/users4/xli/ai-psychiatrist/datasets/daic_woz_dataset/train_split_Depression_AVEC2017.csv")

# Reformatting transcript data to be a string with speaker name + text
patient_transcript = '\n'.join(test_transcript['speaker'] + ': ' + test_transcript['value'])

print(patient_transcript[:677] + "\n")
print(train_phq8_dataset.iloc[2])

Ellie: hi i'm ellie thanks for coming in today i was created to talk to people in a safe and secure environment think of me as a friend i don't judge i can't i'm a computer  
Ellie: i'm here to learn about people and would love to learn about you i'll ask a few questions to get us started and please feel free to tell me anything your answers are totally confidential 
Ellie: how are you doing today
Participant: i'm doing alright
Ellie: that's good
Ellie: where are you from originally
Participant: uh originally i'm from california
Participant: uh born in glendale
Ellie: really
Ellie: nice
Ellie: how do you like your living situation
Participant: i'm not too happy with it

Participant_ID        305.0
PHQ8_Binary             0.0
PHQ8_Score              7.0
Gender                  1.0
PHQ8_NoInterest         0.0
PHQ8_Depressed          1.0
PHQ8_Sleep              1.0
PHQ8_Tired              2.0
PHQ8_Appetite           2.0
PHQ8_Failure            1.0
PHQ8_Concentrating      0.0
PHQ8_Moving  

## Qualitative Assessment

In [4]:
qualitative_prompt = f"""Please analyze the following interview transcript and assess the patient's mental health status, focusing on signs of depression, from social and biological aspects. 

Please provide your assessment in a structured format, including the following sections:
1. Overall Impression in <overall> tags: A brief summary of the patient's mental health status.
2. Social Aspects in <social> tags: Observations related to the patient's interpersonal relationships, family situation, and any relevant social interactions that may influence their mental health.
3. Biological Aspects in <biological> tags: Observations related to the patient's physical health, including sleep quality, appetite, physical activity, stress level, and any other biological factors that may influence their mental health.
4. Additional Notes in <notes> tags: Any other relevant observations or comments that do not fit into the above categories.
5. Potential Risk Factors in <risks> tags: Any identified risk factors for depression or other mental health issues.

Here is the interview transcript:
{patient_transcript}
"""
# Temp, top_k, and top_p set to more deterministic values
response = requests.post(
  BASE_URL,
  json = {
    "model": model,
    "messages": [{"role": "user", "content": qualitative_prompt}],
    "stream": False,
    "options": {
      "temperature": 0,
      "top_k": 20,
      "top_p": 0.9
    }
  }
)

print(json.dumps(response.json(), indent=2))



{
  "model": "gemma3-optimized:27b",
  "created_at": "2025-07-08T19:16:00.939015947Z",
  "message": {
    "role": "assistant",
    "content": "Here's an analysis of the patient's mental health status, based on the provided interview transcript, structured as requested:\n\n**<overall>**\n\nThe patient presents with indicators of both current and past mental health challenges, specifically a diagnosed bipolar disorder. While currently not in therapy due to financial constraints, they acknowledge the benefit of past therapeutic interventions. They exhibit a complex emotional landscape, marked by periods of irritability, anxiety, and a tendency towards social withdrawal. Despite these challenges, they demonstrate resilience, a desire for self-improvement, and a capacity for positive relationships (particularly with their girlfriend). The patient appears to be functioning, but is clearly carrying a significant emotional burden.\n\n**<social>**\n\n*   **Relationships:** The patient has a clo

## Quantitative Assesment

In [13]:
class PHQ8Scores(BaseModel):
    nointerest: int  # 0-3: Little interest or pleasure in doing things
    depressed: int   # 0-3: Feeling down, depressed, or hopeless
    sleep: int       # 0-3: Trouble falling/staying asleep, or sleeping too much
    tired: int       # 0-3: Feeling tired or having little energy
    appetite: int    # 0-3: Poor appetite or overeating
    failure: int     # 0-3: Feeling bad about self or like a failure
    concentrating: int # 0-3: Trouble concentrating
    moving: int      # 0-3: Moving/speaking slowly or being fidgety/restless

quantitative_prompt = f"""Analyze the following interview transcript and predict PHQ-8 scores. The PHQ-8 measures depression symptoms over the past 2 weeks with scores: 0 = not at all (0-1 days), 1 = several days (2-6 days), 2 = more than half the days (7-11 days), 3 = nearly every day (12-14 days).

Return ONLY a JSON object with these exact keys and integer values 0-3:
- "nointerest": score for little interest or pleasure in activities
- "depressed": score for feeling down, depressed, or hopeless  
- "sleep": score for sleep problems (trouble falling/staying asleep or sleeping too much)
- "tired": score for feeling tired or having little energy
- "appetite": score for appetite changes (poor appetite or overeating)
- "failure": score for negative self-perception or feeling like a failure
- "concentrating": score for trouble concentrating on tasks
- "moving": score for psychomotor changes (moving/speaking slowly or restlessness)

Interview transcript:
{patient_transcript}

Respond with valid JSON only."""

# Most deterministic temp, top_k, and top_p
response = requests.post(
    BASE_URL,
    json={
        "model": model,
        "messages": [{"role": "user", "content": quantitative_prompt}],
        "stream": False,
        "options": {
            "temperature": 0,
            "top_k": 1,
            "top_p": 1.0
        }
    }
)

# Parse and validate the response
try:
    response_data = response.json()
    content = response_data['message']['content']
    
    # Remove markdown code blocks if present
    if content.startswith('```json'):
        content = content.split('```json')[1].split('```')[0].strip()
    elif content.startswith('```'):
        content = content.split('```')[1].split('```')[0].strip()
    
    # Parse the JSON response and validate with Pydantic
    scores_dict = json.loads(content)
    phq8_scores = PHQ8Scores(**scores_dict)
    
    # Extract the 8 PHQ-8 values
    scores_list = [
        phq8_scores.nointerest,
        phq8_scores.depressed,
        phq8_scores.sleep,
        phq8_scores.tired,
        phq8_scores.appetite,
        phq8_scores.failure,
        phq8_scores.concentrating,
        phq8_scores.moving
    ]
    
    # Get ground truth values
    ground_truth = train_phq8_dataset.iloc[2]

    print("Comparison of Predicted vs Ground Truth:")
    print("Metric\t\t\tPredicted\tGround Truth\tDifference")
    print("-" * 65)

    differences = []
    metrics = ['PHQ8_NoInterest', 'PHQ8_Depressed', 'PHQ8_Sleep', 'PHQ8_Tired', 
            'PHQ8_Appetite', 'PHQ8_Failure', 'PHQ8_Concentrating', 'PHQ8_Moving']
    predicted_values = [phq8_scores.nointerest, phq8_scores.depressed, phq8_scores.sleep, 
                    phq8_scores.tired, phq8_scores.appetite, phq8_scores.failure,
                    phq8_scores.concentrating, phq8_scores.moving]

    for metric, pred_val in zip(metrics, predicted_values):
        gt_val = int(ground_truth[metric])
        diff = abs(pred_val - gt_val)
        differences.append(diff)
        print(f"{metric:<23} {pred_val:<12} {gt_val:<15} {diff}")

    avg_difference = sum(differences) / len(differences)
    print("-" * 65)
    print(f"Average Absolute Difference: {avg_difference:.2f}")

except (json.JSONDecodeError, KeyError, ValueError) as e:
    print(f"Error parsing response: {e}")
    print("Raw response:", response.json())

Comparison of Predicted vs Ground Truth:
Metric			Predicted	Ground Truth	Difference
-----------------------------------------------------------------
PHQ8_NoInterest         2            0               2
PHQ8_Depressed          2            1               1
PHQ8_Sleep              1            1               0
PHQ8_Tired              2            2               0
PHQ8_Appetite           0            2               2
PHQ8_Failure            1            1               0
PHQ8_Concentrating      1            0               1
PHQ8_Moving             0            0               0
-----------------------------------------------------------------
Average Absolute Difference: 0.75


In [ ]:
"""Available models:
- gemma3-optimized:27b
- gemma3_optimized:8bit
- gemma3:27b-it-qat
- gemma3:27b
- phi4-optimized:latest
- phi4:latest
- phi4-reasoning:plus
- qwen3_optimized:latest
- qwen3:32b
- llama3.3:latest
- llama4:scout
- mistral3.1-optimized:latest
- mistral-small3.1:latest
- qwq:latest

Notes:
1. The models marked as "optimized" are configured to use multiple GPUs for faster inference.
2. The llama4:maverick model is not available due to its size.
"""

OLLAMA_NODE = "arctrdgn001" # TODO: Change this variable to the node where Ollama is running
BASE_URL = f"http://{OLLAMA_NODE}:11434/api/chat"

model = "gemma3-optimized:27b" # TODO: Change this variable to the model you want to use
message = "What is the capital of France?" # TODO: Change this variable to the message you want to ask the model

response = requests.post(
  BASE_URL,
  json = {
    "model": model,
    "messages": [{"role": "user", "content": message}],
    "stream": False
  }
)

print(json.dumps(response.json(), indent=2))

{
  "model": "gemma3-optimized:27b",
  "created_at": "2025-07-08T17:40:17.231880462Z",
  "message": {
    "role": "assistant",
    "content": "The capital of France is **Paris**. \n\nIt's also the country's most populous city!\n\n\n\n"
  },
  "done_reason": "stop",
  "done": true,
  "total_duration": 34293797307,
  "load_duration": 31815583539,
  "prompt_eval_count": 16,
  "prompt_eval_duration": 578633882,
  "eval_count": 24,
  "eval_duration": 1898829459
}
